Let's analyze the modules with the prefix `test_modules` in the current directory.

In [1]:
from mypy_static_analysis import mypy_static_analysis
from get_module_names_and_file_paths_for_pure_python_project import get_module_names_and_file_paths_for_pure_python_project


project_path = '.'
module_prefix = 'test_modules'


def module_name_starts_with_prefix(
    module_name: str,
    prefix: str
) -> bool:
    module_name_components = module_name.split('.')
    prefix_components = prefix.split('.')

    return module_name_components[:len(prefix_components)] == prefix_components

module_names_to_file_paths = {
    module_name: file_path
    for module_name, file_path
    in get_module_names_and_file_paths_for_pure_python_project(project_path)
    if module_name_starts_with_prefix(module_name, module_prefix)
}

result = mypy_static_analysis(project_path, module_names_to_file_paths)

Any errors?

In [2]:
result.errors

["test_modules/function_class_definition_and_imports.py:1: error: No library stub file for module 'numpy'",
 "test_modules/nested_class_and_nested_async_function.py:1: error: No library stub file for module 'aiohttp'",
 'test_modules/nested_class_and_nested_async_function.py:1: note: (Stub files are from https://github.com/python/typeshed)']

Let's check out the module `test_modules.function_class_definition_and_imports`, which starts with:

```python
import numpy as np

def shell_sort(collection):
    xs = [701, 301, 132, 57, 23, 10, 4, 1]
    for x in xs:
        i = x
        while i < len(collection):
            temp = collection[i]
            j = i
            while j >= x and collection[j - x] > temp:
                collection[j] = collection[j - x]
                j -= x
            collection[j] = temp
            i += 1
    return collection


def set_x():
    global x
    x = 2
    shell_sort([1,2,3,4])
```

In [3]:
result.graph.keys()

dict_keys(['test_modules', 'test_modules.nested_class_and_nested_async_function', 'test_modules.all_syntax_constructs', 'test_modules.multiple_inheritance', 'test_modules.function_class_definition_and_imports', 'builtins', 'asyncio', 're', 'random', 'time', 'typing', 'abc', 'ast', 'types', 'sys', 'asyncio.coroutines', 'asyncio.protocols', 'asyncio.streams', 'asyncio.subprocess', 'asyncio.transports', 'asyncio.futures', 'asyncio.tasks', 'asyncio.base_events', 'asyncio.events', 'asyncio.queues', 'asyncio.locks', 'asyncio.runners', 'enum', '_random', 'collections', 'typing_extensions', '_ast', '_importlib_modulespec', 'importlib.abc', 'os', 'concurrent.futures', 'contextvars', 'concurrent', 'selectors', 'socket', 'ssl', 'collections.abc', 'importlib', 'io', 'os.path', 'posix', 'concurrent.futures._base', 'concurrent.futures.thread', 'concurrent.futures.process', 'codecs', 'mmap', 'multiprocessing.context', 'multiprocessing', 'logging', 'multiprocessing.synchronize', 'multiprocessing.queue

In [4]:
result.graph['test_modules.function_class_definition_and_imports']

In [5]:
result.graph['test_modules.function_class_definition_and_imports'].tree

In [6]:
mypy_file = result.graph['test_modules.function_class_definition_and_imports'].tree

Let's look at its defs.

In [7]:
mypy_file.defs

# Name Resolution

Let's look at the definition for `shell_sort`.

In [8]:
func_def = mypy_file.defs[1]

In [9]:
func_def

In [10]:
func_def.body

In [11]:
func_def.body.body

Let's look at the `xs` in `xs = [701, 301, 132, 57, 23, 10, 4, 1]` and the `xs` in `for x in xs:`

In [12]:
func_def.body.body[0]

In [13]:
func_def.body.body[0].lvalues

In [14]:
func_def.body.body[0].lvalues[0]

In [15]:
func_def.body.body[0].lvalues[0].node

In [16]:
func_def.body.body[1]

In [17]:
func_def.body.body[1].expr

In [18]:
func_def.body.body[1].expr.node

In [19]:
func_def.body.body[0].lvalues[0].node is func_def.body.body[1].expr.node

True

They are referring to the same node!

Let's look at the `len` in `len(collection)`.

In [20]:
func_def.body.body[1].body

In [21]:
func_def.body.body[1].body.body

In [22]:
func_def.body.body[1].body.body[1]

In [23]:
func_def.body.body[1].body.body[1].expr

In [24]:
func_def.body.body[1].body.body[1].expr.operands

In [25]:
func_def.body.body[1].body.body[1].expr.operands[1]

This is pointing at the function call `len(collection)`.

In [26]:
func_def.body.body[1].body.body[1].expr.operands[1].callee

In [27]:
func_def.body.body[1].body.body[1].expr.operands[1].callee.node

`len` resolves to a function definition (from Typeshed)!

In [28]:
func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type

def (typing.Sized) -> builtins.int

We can retrieve its type!

In [29]:
func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type.arg_types

[typing.Sized]

In [30]:
func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type.arg_types[0]

typing.Sized

In [31]:
arg_type = func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type.arg_types[0]

In [32]:
type(arg_type)

mypy.types.Instance

In [33]:
func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type.ret_type

builtins.int

In [34]:
ret_type = func_def.body.body[1].body.body[1].expr.operands[1].callee.node.type.ret_type

In [35]:
type(ret_type)

mypy.types.Instance

What's the definition of `typing.Sized`?

In [36]:
arg_type.type

<TypeInfo typing.Sized>

In [37]:
arg_type.type.names

{'__len__': <mypy.nodes.SymbolTableNode at 0x786a06a8b590>}

Let's look at the class definition here:

```python
class Counter:
    def __init__(self):
        # Instance variable to track counts
        self.count = 0
    global_counter = global_counter + 1 # Shadows name in LHS, but refers to name in RHS
    def update_counter(self):
        # Local import within the function for demonstration
        import time

        # Method variable
        method_counter = 0

        def increment():
            # Use nonlocal to modify method_counter
            nonlocal method_counter
            # Use global to modify the global_counter
            global global_counter

            try:
                # Use the globally imported global_randint function directly
                increment = global_randint(1, 10)
                # Simulate a random error
                if choice([True, False]):
                    raise ValueError("Simulated error")

                method_counter += increment
                global_counter += increment
                self.count += increment

                # Demonstrating the use of the locally imported time module
                time.sleep(1)  # Sleep for 1 second to simulate a delay

                print(f"Method counter incremented by {increment}, total method counter: {method_counter}")
                print(f"Global counter updated to {global_counter}, instance counter: {self.count}")
            
            except ValueError as e:
                print(f"Exception caught: {e}")

        increment()
    print(update_counter)
    print(global_counter) # handles shadowed name
```

In [41]:
class_def = mypy_file.defs[8]

In [42]:
class_def

In [44]:
class_def.defs

In [45]:
class_def.defs.body

Let's look at its `__init__` method.

In [46]:
class_def.defs.body[0]

In [49]:
class_def.defs.body[0].type

In [50]:
class_def.defs.body[0].arguments

In [51]:
class_def.defs.body[0].arguments[0]

In [54]:
class_def.defs.body[0].arguments[0].variable

In [62]:
class_def.defs.body[0].arguments[0].variable.is_self

True

We can statically determine that it is `self`!

In [65]:
class_def.defs.body[0].body

In [66]:
class_def.defs.body[0].body.body

In [67]:
class_def.defs.body[0].body.body[0]

In [68]:
class_def.defs.body[0].body.body[0].lvalues

In [69]:
class_def.defs.body[0].body.body[0].lvalues[0]

In [70]:
class_def.defs.body[0].body.body[0].lvalues[0].node

In [74]:
class_def.defs.body[0].body.body[0].lvalues[0].node.name()

'count'

In [75]:
class_def.defs.body[0].body.body[0].lvalues[0].expr

In [77]:
class_def.defs.body[0].body.body[0].lvalues[0].expr.node